# Robot imitation learning
Objective: a modern tensorflow reimplementation of the code of Rouhollah and Pooya's papers.

## TODOS
    [x] Implement an encoder class (VAE only)
    [ ] Implement an encoder class (VAE-GAN)
    [ ] Implement an encoder class (VGG-11 based)
    [x] Implement a training model for the encoder
    [ ] Implement a controller: direct mapping from z to a
    [ ] Implement the controller: LSTM class with the right kind of output, and skip connections
    [ ] Implement the end-to-end training for controller

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pathlib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Softmax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from IPython import display
import pickle
import logging
import platform
import time
%matplotlib inline

from CVAE_VisualModule import CVAE_VisualModule
from MLP_Controller import MLP_Controller
from Datasets import Datasets
# replace this one with the local one when debugging or developing
# from RobotVisualController import RobotVisualController
import paths

## That abstract implementations of the types of modules. 
Not necessarily needed as an inheritence, this is just duck inheritance

    class VisionModule:
        """The abstract ancestor of all the visual modules."""
        def encode(self, x):
            """Encodes a visual input into an intermediary h representation"""
            print("VisualModel.encode")
            raise Exception("This is an abstract function, should not have been called")

        @staticmethod
        def load(path):
            """Loads the visual model from a specific path"""
            print("VisualModel.load")
            raise Exception("This is an abstract function, should not have been called")

    class ControlModule:
        """The abstract ancestor for all imitation learned control modules"""
        def control(h):
            """Generates an action a from a hidden representation"""
            print("Controller.control")
            raise Exception("This is an abstract function, should not have been called")

        @staticmethod
        def load(path):
            """Loads the visual model from a specific path"""
            print("VisualModel.load")
            raise Exception("This is an abstract function, should not have been called")

In [2]:
import numpy as np
from CVAE_VisualModule import CVAE_VisualModule
from MLP_Controller import MLP_Controller

class RobotVisualController:
    """This class contains the implementation of the robot controller. It has various networks which are
    trained in various setups."""
    
    @staticmethod
    def generate_visual_module(config):
        """Factory method to generate a visual module according to the configuration. 
        While it would be somewhat convenient to pass the config file over, I am going to recreate here 
        the config file, to make the transferred data more visible"""
        name = config["visual_module"]
        if name == "CVAE":
            cvaeconfig = {"latent_dim": config["h_size"], "image_width": config["image_width"], 
                          "image_height": config["image_height"], 
                          "image_color_channels": config["image_color_channels"], 
                          "training_data_dir": config["visual_module_training_data_dir"],
                          "model_path": config["visual_module_model_path"],
                          "epochs_target": config["visual_module_epochs_target"],
                          "load_only": False 
                         }
            # FIXME: the CVAE visual module does not take this in this way
            # FIXME: separate component when I want load only vs training
            # 
            module = CVAE_VisualModule.get_trained_model(cvaeconfig)
            return module
        if name == "CVAE-GAN":
            raise Exception("CVAE-GAN visual module not implemented yet")
        if name == "VGG":
            raise Exception("VGG visual module not implemented yet")
        
    @staticmethod
    def generate_control_module(config, visual_module):
        """Factory method to generate a control module according to the configuration"""
        name = config["control_module"]
        if name == "MLPController":
            controllerconfig = {
                "latent_dim": config["h_size"], "control_dim": config["a_size"], 
                "epochs_target": config["control_module_epochs_target"], 
                "model_path": config["control_module_model_path"], 
                "epochs_target": config["control_module_epochs_target"],                
                "demonstration_control_path": config["control_module_demonstration_control_path"],
                "demonstration_images_dir": config["control_module_demonstration_images_dir"],
                "load_only": False }
            controller = MLP_Controller.get_trained_model(controllerconfig, visual_module)
            return controller
        if name == "LSTMController":
            raise Exception("LSTMController not implemented yet")
    
    def __init__(self, config):
        """Creates a robotic initialization controller, and tries to initialize it, if possible"""
        self.config = config
        self.visual_module = self.generate_visual_module(config)
        self.control_module = self.generate_control_module(config, self.visual_module)        
            
    def control(self, image):  
        imagebatch = np.array([image])
        h = self.visual_module.encode(imagebatch)
        a = self.control_module.predict(h)
        return a

## Create the visual controller and train it on the small local dataset

In [3]:
control_path = pathlib.Path(paths.demonstration_dir, "10090.txt")
pictures_dir = pathlib.Path(paths.demonstration_dir, "pictures")


config = {
        "load_only": False,
        "visual_module": "CVAE", 
        "visual_module_model_path": paths.visual_module_model_path,
        "visual_module_training_data_dir": paths.unsupervised_dir,
        "visual_module_epochs_target": 100,
        "control_module": "MLPController", 
        "control_module_model_path": paths.controller_model_path,
        "control_module_demonstration_control_path": control_path,
        "control_module_demonstration_images_dir": pictures_dir,
        "control_module_epochs_target": 100,
        "h_size": 50, "a_size": 7, 
        "image_width": 32, "image_height": 32,
        "image_color_channels": 3
}
rvc = RobotVisualController(config)

INFO:root:model loaded from /home/lboloni/Documents/HackingWork/__Temporary/2021-05-01-RobotImitationLearning/cvae
INFO:root:Model fully trained to the required number of epochs 100
INFO:root:model loaded from /home/lboloni/Documents/HackingWork/__Temporary/2021-05-01-RobotImitationLearning/mlp
INFO:root:Model fully trained to the required number of epochs 100


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 30)                1530      
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 217       
Total params: 1,747
Trainable params: 1,747
Non-trainable params: 0
_________________________________________________________________


## Test whether the controller works the way it is supposed to

In [5]:
control_path = pathlib.Path(paths.demonstration_dir, "10090.txt")
pictures_dir = pathlib.Path(paths.demonstration_dir, "pictures")

ds = Datasets.create_image_control_demonstration_dataset(
    control_path, pictures_dir)
for element in ds.take(5):
    image = element['image']
    a = element['control']
    #plt.imshow(image, extent=[0, 32, 0, 32])
    #display.display(plt.gcf())
    # display.clear_output(wait=False)
    #display.clear_output(wait=True)
    ahat = rvc.control(image)[0]
    loss = tf.keras.losses.MSE(a, ahat)
    print(f"loss={loss}\n a={a}\n a^ {ahat}")
    #plt.plot(output[0][2:])
    #plt.plot(a['control'][2:])
    #time.sleep(1)
    

INFO:root:Started creating image control demonstration dataset from /home/lboloni/Documents/HackingWork/2021-05-01-RobotImitationLearning/data/demonstration-32-task-3001-10090/10090.txt
INFO:root:Done creating image control demonstration dataset, total size 999


loss=0.0016012295382097363
 a=[0.         1.         0.73699999 0.81950003 0.66350001 0.4745
 0.68800002]
 a^ [0.01198746 0.9271904  0.7237102  0.7888837  0.6955532  0.52630156
 0.7186413 ]
loss=0.0009082158794626594
 a=[0.         1.         0.73699999 0.81950003 0.66350001 0.4745
 0.68800002]
 a^ [0.01634654 1.0275185  0.7479505  0.8156893  0.69755065 0.53444207
 0.7091211 ]
loss=0.0006901562446728349
 a=[0.         1.         0.73699999 0.81950003 0.66350001 0.4745
 0.68800002]
 a^ [0.01652413 1.0154225  0.74489474 0.80528843 0.6997137  0.52180946
 0.7105007 ]
loss=0.0009974909480661154
 a=[0.         1.         0.73750001 0.81650001 0.6595     0.47350001
 0.6875    ]
 a^ [0.01607099 1.0165257  0.75343835 0.81465375 0.7059434  0.53515965
 0.7028206 ]
loss=0.0008161106379702687
 a=[0.         1.         0.73750001 0.81650001 0.6595     0.47350001
 0.6875    ]
 a^ [0.02422418 1.0031967  0.74009633 0.8201432  0.7014524  0.5248819
 0.7138748 ]


## Trying to train on the real dataset

In [6]:
control_path = pathlib.Path(paths.demonstrations_rouhi_dir, "task-3001", "10090.txt")
pictures_dir = pathlib.Path(paths.demonstrations_rouhi_dir, "task-3001", "10090", "camera-1")
unsupervised_dir = pathlib.Path(paths.demonstrations_rouhi_dir, "task-3001", "10090", "camera-1")


config = {
        "load_only": False,
        "visual_module": "CVAE", 
        "visual_module_model_path": paths.visual_module_model_path,
        "visual_module_training_data_dir": unsupervised_dir,
        "visual_module_epochs_target": 100,
        "control_module": "MLPController", 
        "control_module_model_path": paths.controller_model_path,
        "control_module_demonstration_control_path": control_path,
        "control_module_demonstration_images_dir": pictures_dir,
        "control_module_epochs_target": 100,
        "h_size": 50, "a_size": 7, 
        "image_width": 32, "image_height": 32,
        "image_color_channels": 3
}
rvc = RobotVisualController(config)

INFO:root:Proceed to train the model
INFO:root:At image: 5000
INFO:root:At image: 10000
INFO:root:At image: 15000
INFO:root:At image: 20000
INFO:root:At image: 25000
INFO:root:At image: 30000
INFO:root:At image: 35000
INFO:root:At image: 40000
INFO:root:At image: 45000
INFO:root:At image: 50000
INFO:root:At image: 55000


2130.85889
2129.50732
2128.91846
2128.81421
2127.68604
2127.96387
2127.00513
2126.646
2125.95776
2125.53955
2124.8291
2124.75
2123.84717
2123.38672
2122.68311
2122.38086
2122.21631
2121.58813
2120.6582
2120.80225
2120.03564
2119.45801
2118.50586
2118.08276
2117.49658
2116.86035
2116.68018
2115.6731
2115.15503
2114.78809
2114.22
2112.72461
2112.46973
2111.15625
2110.52124
2109.79248
2109.63574
2108.67896
2107.20068
2105.82104
2105.59961
2104.36621
2103.49658
2102.56177
2101.27
2101.00293
2099.88477
2098.4624
2097.99121
2096.41309
2096.66113
2092.44141
2093.23413
2090.52881
2090.88281
2088.31323
2088.3125
2087.1272
2083.31445
2083.02539
2082.01392
2079.62939
2077.88428
2078.44775
2078.16284
2077.07031
2075.14697
2072.24268
2069.99512
2069.89868
2066.91846
2066.57031
2065.62231
2065.87671
2060.15063
2061.33374
2058.87476
2059.63159
2057.14893
2055.29907
2052.02417
2052.75171
2054.1626
2049.73926
2049.59619
2051.98291
2049.67969
2045.5011
2043.49902
2046.9729
2042.46558
2042.62427
2040.196

1927.82544
1932.01575
1921.83911
1934.53613
1938.73511
1920.82019
1939.65979
1945.43774
1933.98059
1936.96716
1919.49097
1949.1792
1933.92615
1940.00659
1938.10754
1940.62988
1931.9751
1934.48633
1942.84058
1933.20422
1924.95068
1933.07239
1935.07495
1941.4231
1920.88696
1924.39209
1928.07263
1929.50024
1957.23364
1946.62012
1926.18872
1928.94165
1929.77979
1937.82068
1932.99854
1940.82568
1928.08789
1936.75037
1927.67529
1946.49438
1931.99707
1933.80444
1933.33691
1928.15698
1929.35779
1926.89856
1940.38232
1939.4436
1941.54102
1927.21704
1938.77869
1932.20007
1926.85205
1937.21973
1918.93665
1932.41052
1940.2959
1926.94946
1927.7865
1943.16614
1925.01978
1932.29028
1927.80859
1944.86084
1952.23694
1927.03247
1936.1228
1933.67432
1932.94055
1925.4668
1929.55652
1944.49146
1921.6228
1938.88721
1928.88843
1925.33
1926.08423
1923.85791
1923.45532
1930.52905
1919.48975
1942.8739
1938.94287
1934.75171
1942.48511
1936.34436
1917.96338
1929.60681
1928.55969
1925.42065
1919.97266
1927.4137
19

1920.06787
1916.91174
1912.59814
1902.85986
1910.33337
1930.63879
1910.10303
1933.18848
1911.32202
1906.51172
1915.08435
1915.24683
1911.12598
1911.12646
1912.83716
1927.12671
1903.34363
1919.76294
1914.35254
1913.60974
1907.9552
1915.72156
1921.76855
1904.80444
1899.94458
1915.98242
1911.62524
1912.78784
1910.25244
1903.97681
1918.4657
1925.04956
1912.44116
1919.94177
1912.07324
1906.76196
1915.37305
1908.99805
1911.69067
1900.03271
1925.08423
1903.52051
1907.95972
1907.68579
1906.2478
1898.26807
1910.70923
1906.29028
1908.4585
1910.91846
1913.91309
1923.91919
1926.59521
1913.46704
1924.27197
1901.04272
1904.67261
1902.81934
1909.31152
1915.37366
1925.77539
1915.15784
1916.80542
1920.4209
1919.75183
1899.96631
1904.69629
1907.71167
1903.69214
1908.94031
1920.64136
1905.10693
1910.48633
1898.8125
1903.75562
1903.84546
1909.29492
1915.73486
1907.45947
1908.20911
1905.27673
1917.78918
1907.31079
1913.44543
1914.37732
1913.72192
1923.28735
1902.70154
1916.0531
1917.48022
1910.25244
1925.2

1905.80176
1922.7334
1904.62817
1911.1853
1891.0824
1909.16064
1904.82849
1895.80505
1916.41431
1918.94019
1906.25647
1892.34131
1901.11218
1908.85864
1920.45142
1914.9021
1894.64478
1907.84753
1908.24976
1901.12598
1910.20166
1896.12573
1916.03882
1900.00916
1925.17529
1909.86084
1910.23193
1898.33655
1916.63977
1899.53271
1925.4541
1907.70325
1911.05542
1907.16516
1927.3396
1903.67432
1923.60278
1907.28247
1905.77173
1907.8894
1898.77747
1925.05396
1911.07959
1915.34558
1889.90051
1900.72485
1917.40088
1903.53149
1909.37695
1913.448
1912.63403
1906.39795
1903.50293
1912.08142
1902.97046
1893.86279
1919.10156
1912.96021
1907.37317
1900.52759
1903.77087
1916.48413
1903.8186
1910.7262
1903.24243
1894.85938
1902.73828
1900.47009
1902.83447
1914.42993
1907.02661
1933.34473
1899.53027
1917.31189
1914.55457
1903.88184
1910.98877
1893.50562
1908.37
1896.35657
1902.36475
1909.88892
1894.17749
1897.15991
1901.67346
1907.53174
1914.85645
1909.50305
1897.71106
1912.05066
1899.41895
1912.5553
189

...

1888.86072
1893.82141
1900.87244
1891.8678
1875.39673
1881.66406
1887.6394
1888.79907
1887.43872
1883.49731
1889.49146
1883.04858
1877.19604
1884.08667
1872.64124
1880.61938
1887.49353
1880.09937
1889.43286
1874.3114
1874.72485
1899.62207
1884.68958
1903.50903
1886.3689
1885.40991
1892.60693
1882.42871
1893.8009
1894.56
1879.56274
1885.96069
1871.62708
1877.07373
1877.26025
1884.84448
1895.51038
1879.87646
1873.20691
1898.00635
1894.41858
1891.88843
1893.70044
1878.62378
1886.27051
1883.06042
1893.96338
1894.58496
1885.41687
1886.67432
1889.34229
1907.8269
1895.4646
1882.38037
1886.72656
1879.99756
1889.54468
1897.70569
1885.25806
1890.57251
1894.40479
1895.09534
1884.03174
1872.75647
1871.99072
1893.1853
1908.11328
1876.32446
1890.83643
1887.07715
1890.91406
1868.20471
1888.39771
1871.88477
1874.22351
1880.51074
1879.97461
1885.82129
1880.5542
1872.69922
1880.3877
1880.96777
1881.52881
1883.41772
1900.40173
1881.61743
1886.50244
1890.05737
1881.0343
1884.31934
1871.61658
1880.08679
18

1898.21936
1895.11035
1875.98
1892.14697
1887.7373
1891.39551
1890.77783
1887.45593
1882.39771
1880.41577
1877.69556
1888.82788
1878.31702
1887.30066
1896.42944
1888.20569
1884.56641
1883.79749
1886.80737
1875.0874
1876.27258
1884.22729
1877.797
1894.31287
1875.72876
1875.58789
1888.18982
1885.68652
1887.3916
1872.2207
1882.61072
1883.93652
1899.00635
1877.50964
1886.53992
1897.68042
1887.54224
1902.20886
1894.59155
1881.20837
1873.67065
1890.61841
1889.35
1897.05334
1871.02332
1870.31274
1898.02466
1870.95422
1891.19971
1880.75037
1885.96045
1884.13794
1898.35535
1884.27551
1887.22046
1886.86218
1874.88818
1882.41846
1878.97241
1887.06641
1898.28027
1889.46729
1878.32471
1881.96704
1875.98865
1887.17896
1899.60168
1886.11597
1900.2356
1882.15649
1890.40649
1883.49902
1879.42285
1877.75378
1888.93909
1884.17676
1882.6272
1886.20227
1896.75195
1867.56653
1873.76
1902.32129
1899.24866
1889.79797
1887.44019
1879.92725
1878.73828
1873.47778
1884.47314
1881.8418
1881.83398
1866.7157
1887.76

1889.55273
1889.58728
1878.25745
1879.57422
1881.74194
1883.29785
1898.30908
1876.52393
1884.46326
1903.17395
1882.07861
1893.70459
1883.8252
1888.42407
1881.09119
1894.47571
1880.96948
1885.82556
1899.42065
1886.62012
1897.34888
1882.37378
1885.92383
1880.71509
1878.1842
1881.31079
1885.07861
1882.76489
1890.41284
1889.37683
1874.02759
1889.77808
1885.28552
1887.10596
1884.98315
1891.2699
1870.21
1885.26526
1878.34814
1872.63989
1901.16602
1886.52856
1876.54395
1869.47827
1895.41357
1888.26489
1877.78967
1875.56909
1886.67969
1886.47803
1874.1554
1890.02222
1896.13452
1888.2229
1890.79529
1876.45435
1892.81311
1886.51929
1878.91162
1872.72534
1884.42749
1882.78101
1882.30078
1884.18689
1880.83716
1883.84802
1895.38379
1893.72803
1879.58215
1894.24707
1886.57971
1879.86975
1887.19385
1888.2074
1881.2002
1876.15857
1870.46179
1876.19702
1885.97095
1889.39697
1887.20435
1891.60181
1887.41919
1877.06104
1889.32739
1890.20129
1892.06274
1877.69592
1880.75818
1876.36353
1873.16504
1894.0395

1888.65869
1893.13245
1884.44019
1877.34644
1873.18921
1889.28174
1886.91577
1880.93018
1885.1897
1888.79285


KeyboardInterrupt: 